In [41]:
import scispacy
import spacy
import joblib
import re
import string
import re
from nltk import word_tokenize
from collections import Counter
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.sparse import coo_matrix, hstack

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import BertTokenizer, BertModel
import gensim.downloader as api
from gensim.models import KeyedVectors

import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [42]:
df=pd.read_csv("dfnew.csv")

In [43]:
df['target'] = df['target'].str.lower()
df['New Category'] = df['target'].apply(lambda x: 'Fire' if 'fire' in x.lower() else ('cardiac_arrest' if 'cardiac' in x.lower() else ('heart_attack' if 'heart' in x.lower() else x)))
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'chest pain' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'stroke' if 'stroke' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'injured' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'injury' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'seizure' if 'convulsion/seizures' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose / poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose - poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'choking' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'Major_trauma' if 'fall' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'difficulty breathing' if 'overdose/poisoning' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: '911 cellular hang-up/open/misdial' if '911 hang-up/open/misdial' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: '911 cellular hang-up/open/misdial' if 'duplicate call' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'suspicious person' if 'suspicious person/vehicle' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'cardiac_arrest' in x.lower() else x)

In [44]:
unique_categories_num = df['New Category'].value_counts()
counts_df_full = pd.DataFrame({'Category': unique_categories_num.index, 'Count': unique_categories_num.values})
counts_df_full

,Category,Count
0,Major_trauma,205
1,sick person,200
2,difficulty breathing,131
3,lift assist,88
4,heart_attack,86
...,...,...
80,suspicious mail,1
81,burglary,1
82,reckless driver,1
83,lost property report,1


In [45]:
category_counts = df['New Category'].value_counts()
limit = 59
selected_categories = category_counts[category_counts > limit].index.tolist()
df = df.loc[df['New Category'].isin(selected_categories)]

In [46]:
df['New Category'] = df['New Category'].apply(lambda x: 'heart_attack' if 'stroke' in x.lower() else x)
df['New Category'] = df['New Category'].apply(lambda x: 'others' if x != 'heart_attack' else x)

In [47]:
unique_category_nums = df['New Category'].value_counts()
counts_df_full = pd.DataFrame({'Category': unique_category_nums.index, 'Count': unique_category_nums.values})
counts_df_full

,Category,Count
0,others,760
1,heart_attack,86


In [48]:
df

,audio,text,target,clean_text,key_words_chemical,key_words_disease,New Category
0,2021-01-01/4/CH4_03004448_51.txt,0002.0s 0002.5s SPEAKER_01: Umm\n 0003.3s 000...,chest pain,umm need dont care tanya okay number 740971896...,-,pain pain -,heart_attack
1,2021-01-13/6/CH6_05004302_51.txt,"0011.3s 0013.5s SPEAKER_00: 911, what's the a...",sick person,911 whats address emergency 317 chelsea street...,-,fever headaches -,others
2,2021-03-01/6/CH6_05004634_51.txt,"0010.0s 0012.7s SPEAKER_00: 911, what is the ...",domestic,911 address emergency 79 rock creek drive 79 r...,alcohol -,bleeding -,others
3,2021-03-01/6/CH6_05004634_51.txt,"0010.0s 0012.7s SPEAKER_00: 911, what is the ...",injured person - basic,911 address emergency 79 rock creek drive 79 r...,alcohol -,bleeding -,others
4,2021-04-10/5/CH5_0400486D_51.txt,"0011.0s 0012.9s SPEAKER_01: 911, what's the a...",difficulty breathing,911 whats address emergency hello 91 need ambu...,oxygen oxygen -,death -,others
...,...,...,...,...,...,...,...
1451,2021-06-29/4/CH4_03005299_146.txt,0004.5s 0005.5s SPEAKER_04: She needs to grow...,duplicate call,needs grow 911 whats address emergency address...,-,-,others
1454,2021-06-30/6/CH6_0500510B_146.txt,0002.2s 0005.0s SPEAKER_00: yeah\n 0009.9s 00...,911 cellular hang-up/open/misdial,yeah thank 911 address emergency reported ofte...,-,-,others
1456,2021-06-29/2/CH2_010051FC_146.txt,"0012.7s 0014.9s SPEAKER_01: 911, what's the a...",911 cellular hang-up/open/misdial,911 whats address emergency mother earth turn ...,-,-,others
1459,2021-06-30/5/CH5_04004F99_146.txt,"0011.8s 0013.6s SPEAKER_00: 911, what's the a...",injured person - basic,911 whats address emergency really emergency g...,-,-,others


In [49]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

text = "umm need dont care tanya okay number 7409718962 correct yeah okay whats name karger said old going go car howdy kevin 27 k still outside car didnt leave yet okay yeah actually medics coming see 317 chelsea street department okay dont go anywhere okay im going ask questions history heart problems painting chef located mark starts like right left middle middle sudden gradual started um kinda like gradually ty anybo fades gradually normal okay got hat front neighborhood hazards front neighborhood yeah fault thought 317 chelsea street apartment inside apartment im car okay parked front complex want come yes okay hurry yeah marked route im putting youre front complex car kind vehicle state two door four door us whats fun many doors fordar fordar year seven oh yeah feeling pain anywhere else head short breath yes change breathe move change breathe move thank said pain change breathe move yeah dated vomited community oh god bet police okay right ill go ahead disconnect make contact okay right ill go ahead disconnect make contact bye"
tokens = word_tokenize(text)

bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens)-1)]

trigrams = [(tokens[i], tokens[i+1], tokens[i+2]) for i in range(len(tokens)-2)]

unique_bigrams = list(set(bigrams))
unique_trigrams = list(set(trigrams))

[nltk_data] Downloading package punkt to
[nltk_data]     /users/PAS0536/harshinikavuru26/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
import nltk
from nltk import bigrams

def bigrams_extract(text):
    tokens = nltk.word_tokenize(text)
    bg = list(bigrams(tokens))
    bigram_keywords = [f"{w1} {w2}" for w1, w2 in bg]
    return ", ".join(bigram_keywords)

# Assuming you have a pandas DataFrame called 'df'
df["bigrams"] = df["clean_text"].map(bigrams_extract)

In [51]:
df['bigrams'][0]

'umm need, need dont, dont care, care tanya, tanya okay, okay number, number 7409718962, 7409718962 correct, correct yeah, yeah okay, okay whats, whats name, name karger, karger said, said old, old going, going go, go car, car howdy, howdy kevin, kevin 27, 27 k, k still, still outside, outside car, car didnt, didnt leave, leave yet, yet okay, okay yeah, yeah actually, actually medics, medics coming, coming see, see 317, 317 chelsea, chelsea street, street department, department okay, okay dont, dont go, go anywhere, anywhere okay, okay im, im going, going ask, ask questions, questions history, history heart, heart problems, problems painting, painting chef, chef located, located mark, mark starts, starts like, like right, right left, left middle, middle middle, middle sudden, sudden gradual, gradual started, started um, um kinda, kinda like, like gradually, gradually ty, ty anybo, anybo fades, fades gradually, gradually normal, normal okay, okay got, got hat, hat front, front neighborh

In [52]:
from nltk import ngrams

def trigrams_extract(text):
    tokens = nltk.word_tokenize(text)
    tg = list(ngrams(tokens, 3))
    trigram_keywords = [f"{w1} {w2} {w3}" for w1, w2, w3 in tg]
    return ", ".join(trigram_keywords)

# Assuming you have a pandas DataFrame called 'df'
df["trigrams"] = df["clean_text"].map(trigrams_extract)

In [53]:
df['trigrams'][0]

'umm need dont, need dont care, dont care tanya, care tanya okay, tanya okay number, okay number 7409718962, number 7409718962 correct, 7409718962 correct yeah, correct yeah okay, yeah okay whats, okay whats name, whats name karger, name karger said, karger said old, said old going, old going go, going go car, go car howdy, car howdy kevin, howdy kevin 27, kevin 27 k, 27 k still, k still outside, still outside car, outside car didnt, car didnt leave, didnt leave yet, leave yet okay, yet okay yeah, okay yeah actually, yeah actually medics, actually medics coming, medics coming see, coming see 317, see 317 chelsea, 317 chelsea street, chelsea street department, street department okay, department okay dont, okay dont go, dont go anywhere, go anywhere okay, anywhere okay im, okay im going, im going ask, going ask questions, ask questions history, questions history heart, history heart problems, heart problems painting, problems painting chef, painting chef located, chef located mark, locat

In [54]:
df

,audio,text,target,clean_text,key_words_chemical,key_words_disease,New Category,bigrams,trigrams
0,2021-01-01/4/CH4_03004448_51.txt,0002.0s 0002.5s SPEAKER_01: Umm\n 0003.3s 000...,chest pain,umm need dont care tanya okay number 740971896...,-,pain pain -,heart_attack,"umm need, need dont, dont care, care tanya, ta...","umm need dont, need dont care, dont care tanya..."
1,2021-01-13/6/CH6_05004302_51.txt,"0011.3s 0013.5s SPEAKER_00: 911, what's the a...",sick person,911 whats address emergency 317 chelsea street...,-,fever headaches -,others,"911 whats, whats address, address emergency, e...","911 whats address, whats address emergency, ad..."
2,2021-03-01/6/CH6_05004634_51.txt,"0010.0s 0012.7s SPEAKER_00: 911, what is the ...",domestic,911 address emergency 79 rock creek drive 79 r...,alcohol -,bleeding -,others,"911 address, address emergency, emergency 79, ...","911 address emergency, address emergency 79, e..."
3,2021-03-01/6/CH6_05004634_51.txt,"0010.0s 0012.7s SPEAKER_00: 911, what is the ...",injured person - basic,911 address emergency 79 rock creek drive 79 r...,alcohol -,bleeding -,others,"911 address, address emergency, emergency 79, ...","911 address emergency, address emergency 79, e..."
4,2021-04-10/5/CH5_0400486D_51.txt,"0011.0s 0012.9s SPEAKER_01: 911, what's the a...",difficulty breathing,911 whats address emergency hello 91 need ambu...,oxygen oxygen -,death -,others,"911 whats, whats address, address emergency, e...","911 whats address, whats address emergency, ad..."
...,...,...,...,...,...,...,...,...,...
1451,2021-06-29/4/CH4_03005299_146.txt,0004.5s 0005.5s SPEAKER_04: She needs to grow...,duplicate call,needs grow 911 whats address emergency address...,-,-,others,"needs grow, grow 911, 911 whats, whats address...","needs grow 911, grow 911 whats, 911 whats addr..."
1454,2021-06-30/6/CH6_0500510B_146.txt,0002.2s 0005.0s SPEAKER_00: yeah\n 0009.9s 00...,911 cellular hang-up/open/misdial,yeah thank 911 address emergency reported ofte...,-,-,others,"yeah thank, thank 911, 911 address, address em...","yeah thank 911, thank 911 address, 911 address..."
1456,2021-06-29/2/CH2_010051FC_146.txt,"0012.7s 0014.9s SPEAKER_01: 911, what's the a...",911 cellular hang-up/open/misdial,911 whats address emergency mother earth turn ...,-,-,others,"911 whats, whats address, address emergency, e...","911 whats address, whats address emergency, ad..."
1459,2021-06-30/5/CH5_04004F99_146.txt,"0011.8s 0013.6s SPEAKER_00: 911, what's the a...",injured person - basic,911 whats address emergency really emergency g...,-,-,others,"911 whats, whats address, address emergency, e...","911 whats address, whats address emergency, ad..."


In [68]:
word_counts_heart = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category'] == 'heart_attack':
        clean_text = row['clean_text']
        tokens = word_tokenize(clean_text)
        for token in tokens:
            word_counts_heart[token] += 1

sorted_words_heart = sorted(word_counts_heart.items(), key=lambda x: x[1], reverse=True)

In [69]:
sorted_words_dict_heart = {}
for word, count in sorted_words_heart:
    sorted_words_dict_heart[word] = count

In [70]:
df_word_frequency_heart = pd.DataFrame.from_dict(sorted_words_dict_heart, orient='index', columns=['Frequency'])
df_word_frequency_heart = df_word_frequency_heart.sort_values('Frequency', ascending=False)
df_word_frequency_heart.reset_index(inplace=True)
df_word_frequency_heart.columns = ['Words', 'Frequency']
print(df_word_frequency_heart)

                    Words  Frequency
0                    okay        773
1                    pain        282
2                      im        242
3                   right        234
4                    yeah        195
...                   ...        ...
2399            materials          1
2400  coffeegroundlooking          1
2401             chewable          1
2402              onscene          1
2403                greet          1

[2404 rows x 2 columns]


In [71]:
df_word_frequency_heart.to_csv("df_word_frequency_heart.csv")
df_word_frequency_heart.head(50)

,Words,Frequency
0,okay,773
1,pain,282
2,im,242
3,right,234
4,yeah,195
5,like,181
6,chest,169
7,hes,169
8,get,167
9,heart,166


In [76]:
word_counts_others = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category'] == 'others':
        clean_text = row['clean_text']
        tokens = word_tokenize(clean_text)
        for token in tokens:
            word_counts_others[token] += 1

sorted_words_others = sorted(word_counts_others.items(), key=lambda x: x[1], reverse=True)

In [77]:
sorted_words_dict_others = {}
for word, count in sorted_words_others:
    sorted_words_dict_others[word] = count

In [78]:
df_word_frequency_others = pd.DataFrame.from_dict(sorted_words_dict_others, orient='index', columns=['Frequency'])
df_word_frequency_others = df_word_frequency_others.sort_values('Frequency', ascending=False)
df_word_frequency_others.reset_index(inplace=True)
df_word_frequency_others.columns = ['Words', 'Frequency']

In [79]:
df_word_frequency_others.to_csv("df_others_frequency_heart.csv")
df_word_frequency_others.head(50)

,Words,Frequency
0,okay,5732
1,right,1897
2,im,1816
3,whats,1656
4,yeah,1651
5,hes,1582
6,get,1547
7,know,1478
8,shes,1375
9,like,1247


In [55]:
from collections import defaultdict
bigram_counts_heart = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category']=='heart_attack':
        clean_text = row['clean_text']
        tokens = nltk.word_tokenize(clean_text)
        row_bigrams = list(bigrams(tokens))
        for bigram in row_bigrams:
            bigram_counts_heart[bigram] += 1

sorted_bigrams_heart = sorted(bigram_counts_heart.items(), key=lambda x: x[1], reverse=True)

In [56]:
bigram_frequency_dict_heart = {}
for bigram, count in sorted_bigrams_heart:
    bigram_frequency_dict_heart[bigram] = count

In [57]:
bigram_frequency_dict_heart

{('chest', 'pain'): 84,
 ('phone', 'number'): 66,
 ('okay', 'okay'): 66,
 ('address', 'emergency'): 65,
 ('go', 'ahead'): 48,
 ('heart', 'rate'): 45,
 ('back', 'pain'): 44,
 ('im', 'going'): 43,
 ('anywhere', 'else'): 42,
 ('dont', 'know'): 41,
 ('im', 'sorry'): 39,
 ('whats', 'address'): 38,
 ('rapid', 'heart'): 37,
 ('okay', 'whats'): 36,
 ('pain', 'anywhere'): 36,
 ('911', 'whats'): 34,
 ('short', 'breath'): 32,
 ('heart', 'attack'): 31,
 ('history', 'heart'): 30,
 ('whats', 'phone'): 30,
 ('911', 'address'): 30,
 ('pain', 'change'): 28,
 ('whats', 'going'): 28,
 ('okay', 'right'): 27,
 ('whats', 'name'): 26,
 ('tell', 'exactly'): 26,
 ('little', 'bit'): 26,
 ('heart', 'problems'): 25,
 ('yes', 'okay'): 25,
 ('sweating', 'profusely'): 25,
 ('yeah', 'okay'): 24,
 ('covid', 'symptoms'): 24,
 ('okay', 'thank'): 22,
 ('call', 'back'): 22,
 ('pain', 'located'): 22,
 ('feel', 'like'): 22,
 ('okay', 'old'): 21,
 ('front', 'door'): 21,
 ('feel', 'pain'): 21,
 ('breathe', 'move'): 20,
 ('pai

In [58]:
df_bigram_frequency_heart = pd.DataFrame.from_dict(bigram_frequency_dict_heart, orient='index', columns=['Frequency'])
df_bigram_frequency_heart = df_bigram_frequency_heart.sort_values('Frequency', ascending=False)
df_bigram_frequency_heart.reset_index(inplace=True)
df_bigram_frequency_heart.columns = ['Bigram', 'Frequency']
print(df_bigram_frequency_heart)

                     Bigram  Frequency
0             (chest, pain)         84
1              (okay, okay)         66
2           (phone, number)         66
3      (address, emergency)         65
4               (go, ahead)         48
...                     ...        ...
10439      (daughter, okay)          1
10440      (wake, daughter)          1
10441          (want, wake)          1
10442        (second, dont)          1
10443          (ok, sounds)          1

[10444 rows x 2 columns]


In [59]:
df_bigram_frequency_heart.head(50)
df_bigram_frequency_heart.to_csv("df_bigram_frequency_heart.csv")

In [60]:
df_bigram_frequency_heart.head(50)

,Bigram,Frequency
0,"(chest, pain)",84
1,"(okay, okay)",66
2,"(phone, number)",66
3,"(address, emergency)",65
4,"(go, ahead)",48
5,"(heart, rate)",45
6,"(back, pain)",44
7,"(im, going)",43
8,"(anywhere, else)",42
9,"(dont, know)",41


In [61]:
from collections import defaultdict
bigram_counts_others = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category']=='others':
        clean_text = row['clean_text']
        tokens = nltk.word_tokenize(clean_text)
        row_bigrams = list(bigrams(tokens))
        for bigram in row_bigrams:
            bigram_counts_others[bigram] += 1

sorted_bigrams_others = sorted(bigram_counts_others.items(), key=lambda x: x[1], reverse=True)
bigram_frequency_dict_others = {}
for bigram, count in sorted_bigrams_others:
    bigram_frequency_dict_others[bigram] = count
df_bigram_frequency_others = pd.DataFrame.from_dict(bigram_frequency_dict_others, orient='index', columns=['Frequency'])
df_bigram_frequency_others = df_bigram_frequency_others.sort_values('Frequency', ascending=False)
df_bigram_frequency_others.reset_index(inplace=True)
df_bigram_frequency_others.columns = ['Bigram', 'Frequency']
df_bigram_frequency_others.to_csv("df_bigram_frequency_others.csv")

In [62]:
df_bigram_frequency_others.head(50)

,Bigram,Frequency
0,"(address, emergency)",638
1,"(phone, number)",621
2,"(dont, know)",477
3,"(okay, okay)",474
4,"(whats, address)",406
5,"(911, whats)",373
6,"(okay, right)",350
7,"(go, ahead)",330
8,"(okay, whats)",321
9,"(whats, going)",297


In [63]:
from collections import defaultdict

trigram_counts_others = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category'] == 'others':
        clean_text = row['clean_text']
        tokens = nltk.word_tokenize(clean_text)
        row_trigrams = list(ngrams(tokens, 3))
        for trigram in row_trigrams:
            trigram_counts_others[trigram] += 1

sorted_trigrams_others = sorted(trigram_counts_others.items(), key=lambda x: x[1], reverse=True)
trigram_frequency_dict_others = {}
for trigram, count in sorted_trigrams_others:
    trigram_frequency_dict_others[trigram] = count

df_trigram_frequency_others = pd.DataFrame.from_dict(trigram_frequency_dict_others, orient='index', columns=['Frequency'])
df_trigram_frequency_others = df_trigram_frequency_others.sort_values('Frequency', ascending=False)
df_trigram_frequency_others.reset_index(inplace=True)
df_trigram_frequency_others.columns = ['Trigram', 'Frequency']
df_trigram_frequency_others.to_csv("df_trigram_frequency_others.csv", index=False)

In [64]:
df_trigram_frequency_others.head(50)

,Trigram,Frequency
0,"(whats, address, emergency)",347
1,"(911, whats, address)",338
2,"(whats, phone, number)",264
3,"(911, address, emergency)",259
4,"(give, us, call)",164
5,"(us, call, back)",156
6,"(tell, exactly, whats)",143
7,"(okay, okay, thank)",104
8,"(phone, number, case)",102
9,"(number, youre, calling)",97


In [65]:
from collections import defaultdict

trigram_counts_heart = defaultdict(int)

for index, row in df.iterrows():
    if row['New Category'] == 'others':
        clean_text = row['clean_text']
        tokens = nltk.word_tokenize(clean_text)
        row_trigrams = list(ngrams(tokens, 3))
        for trigram in row_trigrams:
            trigram_counts_heart[trigram] += 1

sorted_trigrams_heart = sorted(trigram_counts_heart.items(), key=lambda x: x[1], reverse=True)
trigram_frequency_dict_heart = {}
for trigram, count in sorted_trigrams_heart:
    trigram_frequency_dict_heart[trigram] = count

df_trigram_frequency_heart = pd.DataFrame.from_dict(trigram_frequency_dict_heart, orient='index', columns=['Frequency'])
df_trigram_frequency_heart = df_trigram_frequency_heart.sort_values('Frequency', ascending=False)
df_trigram_frequency_heart.reset_index(inplace=True)
df_trigram_frequency_heart.columns = ['Trigram', 'Frequency']
df_trigram_frequency_heart.to_csv("df_trigram_frequency_heart.csv", index=False)

In [66]:
df_trigram_frequency_heart.head(50)

,Trigram,Frequency
0,"(whats, address, emergency)",347
1,"(911, whats, address)",338
2,"(whats, phone, number)",264
3,"(911, address, emergency)",259
4,"(give, us, call)",164
5,"(us, call, back)",156
6,"(tell, exactly, whats)",143
7,"(okay, okay, thank)",104
8,"(phone, number, case)",102
9,"(number, youre, calling)",97
